<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>LangChain for Generative AI</h1>
<h1>ChatBot</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import langchain

from langchain.prompts import PromptTemplate
from langchain.document_loaders import GutenbergLoader

from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories.in_memory import ChatMessageHistory

from langchain.schema import messages_from_dict, messages_to_dict

from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType

from langchain.chains import LLMChain, ConversationalRetrievalChain, ConversationChain
from langchain.chains import RetrievalQA

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

import langchain_openai
from langchain_openai import ChatOpenAI

import tempfile

import watermark

%load_ext watermark
%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.12.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: cc526109102c100be511b85d0872c4bdfd674d94

matplotlib      : 3.8.0
numpy           : 1.26.4
pandas          : 2.1.4
langchain       : 0.2.2
langchain_openai: 0.1.8
watermark       : 2.4.3



Load default figure style

In [3]:
plt.style.use('./d4sci.mplstyle')

# Start

In [4]:
cache_dir = "./cache"

In [5]:
loader = GutenbergLoader(
    "https://www.gutenberg.org/cache/epub/1513/pg1513.txt"
)

document = loader.load()

extrait = ' '.join(document[0].page_content.split()[:100])
display(extrait + " .......")

'The Project Gutenberg eBook of Romeo and Juliet This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook. Title: Romeo and Juliet Author: William Shakespeare Release date: November .......'

In [6]:
text_splitter = CharacterTextSplitter(
    chunk_size=1024, # Each chunk is of size 1024
    chunk_overlap=256 # Neigboring chunks overlap by 256 characters
) 

texts = text_splitter.split_documents(document)

In [7]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"

embeddings = HuggingFaceEmbeddings(
    model_name=model_name, 
    cache_folder=cache_dir
)  # Use a pre-cached model

/opt/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
vectordb = Chroma.from_documents(
    texts, 
    embeddings, 
    persist_directory=cache_dir
)

In [9]:
question = "Romeo!"

docs = vectordb.similarity_search(question, k=2)

# Check the length of the document
print(len(docs))

# Check the content of the first document
print(docs[0].page_content)
print("="*20)
print(docs[1].page_content)

2
That fair for which love groan’d for and would die,


With tender Juliet match’d, is now not fair.


Now Romeo is belov’d, and loves again,


Alike bewitched by the charm of looks;


But to his foe suppos’d he must complain,


And she steal love’s sweet bait from fearful hooks:


Being held a foe, he may not have access


To breathe such vows as lovers use to swear;


And she as much in love, her means much less


To meet her new beloved anywhere.


But passion lends them power, time means, to meet,


Tempering extremities with extreme sweet.





 [_Exit._]





SCENE I. An open place adjoining Capulet’s Garden.





 Enter Romeo.





ROMEO.


Can I go forward when my heart is here?


Turn back, dull earth, and find thy centre out.





 [_He climbs the wall and leaps down within it._]





 Enter Benvolio and Mercutio.





BENVOLIO.


Romeo! My cousin Romeo! Romeo!





MERCUTIO.


He is wise,


And on my life hath stol’n him home to bed.





BENVOLIO.
That fair for which love g

Create a wrapper around the functionality of our vector database so we can search for similar documents in the vectorstore

In [10]:
retriever = vectordb.as_retriever()

In [11]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

In [12]:
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [13]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [14]:
query = "What is Juliets family?"

query_results_venice = qa.invoke(query)
print("#" * 12)
query_results_venice['result']

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


############


"Juliet's family is the Capulets. Thanks for asking!"

In [15]:
query = "What happens to Romeo and Juliet?"
query_results_romeo = qa.invoke(query)
print("#" * 12)
query_results_romeo['result']

############


'Romeo and Juliet both die as a result of their tragic love story. Thanks for asking!'

In [16]:
query = "Who is Mercutio?"
query_results_romeo = qa.invoke(query)
print("#" * 12)
query_results_romeo['result']

############


'Mercutio is a character in William Shakespeare\'s play "Romeo and Juliet." He is a close friend of Romeo and is known for his wit and humor. Thanks for asking!'

In [17]:
query = "Does Romeo live?"
qa_chain_docs = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       # Return source documents
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain_docs({"query": question})
result["result"]

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Romeo is in love with Juliet, but they are considered foes and cannot openly express their love. However, their passion gives them the means to meet and be together. Thanks for asking!'

In [18]:
len(result['source_documents'])

4

In [19]:
print(result['source_documents'][3].page_content)

That fair for which love groan’d for and would die,


With tender Juliet match’d, is now not fair.


Now Romeo is belov’d, and loves again,


Alike bewitched by the charm of looks;


But to his foe suppos’d he must complain,


And she steal love’s sweet bait from fearful hooks:


Being held a foe, he may not have access


To breathe such vows as lovers use to swear;


And she as much in love, her means much less


To meet her new beloved anywhere.


But passion lends them power, time means, to meet,


Tempering extremities with extreme sweet.





 [_Exit._]





SCENE I. An open place adjoining Capulet’s Garden.





 Enter Romeo.





ROMEO.


Can I go forward when my heart is here?


Turn back, dull earth, and find thy centre out.





 [_He climbs the wall and leaps down within it._]





 Enter Benvolio and Mercutio.





BENVOLIO.


Romeo! My cousin Romeo! Romeo!





MERCUTIO.


He is wise,


And on my life hath stol’n him home to bed.





BENVOLIO.


<center>
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>